In [1]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews, Sort, reviews_all
 
import pandas as pd  # Pandas untuk manipulasi dan analisis data
pd.options.mode.chained_assignment = None  # Menonaktifkan peringatan chaining
import numpy as np  # NumPy untuk komputasi numerik
seed = 0
np.random.seed(seed)  # Mengatur seed untuk reproduktibilitas
import matplotlib.pyplot as plt  # Matplotlib untuk visualisasi data
import seaborn as sns  # Seaborn untuk visualisasi data statistik, mengatur gaya visualisasi
from sklearn.metrics import accuracy_score
 
import datetime as dt  # Manipulasi data waktu dan tanggal
import re  # Modul untuk bekerja dengan ekspresi reguler
import string  # Berisi konstanta string, seperti tanda baca
from nltk.tokenize import word_tokenize  # Tokenisasi teks
from nltk.corpus import stopwords  # Daftar kata-kata berhenti dalam teks
 
# !pip install sastrawi
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory  # Stemming (penghilangan imbuhan kata) dalam bahasa Indonesia
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory  # Menghapus kata-kata berhenti dalam bahasa Indonesia
 
from wordcloud import WordCloud  # Membuat visualisasi berbentuk awan kata (word cloud) dari teks
 
import nltk  # Import pustaka NLTK (Natural Language Toolkit).
nltk.download('punkt')  # Mengunduh dataset yang diperlukan untuk tokenisasi teks.
nltk.download('stopwords')  # Mengunduh dataset yang berisi daftar kata-kata berhenti (stopwords) dalam berbagai bahasa.

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\HP-01\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP-01\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

#### Scraping Dataset
Scraping data adalah cara untuk mengambil informasi dari halaman web dengan otomatis. Hal ini seperti menyapu (scrape) data dari sebuah situs web, mirip seperti cara Anda mengambil informasi dari buku atau majalah dengan membacanya sekilas.
Ketika ingin menganalisis ulasan atau pendapat orang tentang sebuah aplikasi di Google Play Store, kita bisa menggunakan teknik scraping untuk mengumpulkan ulasan-ulasan tersebut secara otomatis. Ini memungkinkan kita untuk memiliki banyak data yang bisa dianalisis lebih lanjut.
Ketika mengambil data dari Google Play Store atau situs web lainnya, kita harus berhati-hati untuk tidak melanggar peraturan. Kadang-kadang, situs web memiliki aturan yang melarang pengambilan data secara otomatis. Jadi, pastikan untuk membaca dan mengikuti aturan-aturan tersebut.

In [2]:
# Mengimpor pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store.
from google_play_scraper import app, reviews_all, Sort
 
# Mengambil semua ulasan dari aplikasi dengan ID 'com.byu.id' di Google Play Store.
# Proses scraping mungkin memerlukan beberapa saat tergantung pada jumlah ulasan yang ada.
scrapreview = reviews_all(
    'com.byu.id',          # ID aplikasi
    lang='id',             # Bahasa ulasan (default: 'en')
    country='id',          # Negara (default: 'us')
    sort=Sort.MOST_RELEVANT, # Urutan ulasan (default: Sort.MOST_RELEVANT)
    count=1000             # Jumlah maksimum ulasan yang ingin diambil
)

Kode di atas kita gunakan untuk mengambil semua ulasan dari sebuah aplikasi di Google Play Store dengan ID 'com.byu.id'. Kita menggunakan pustaka google_play_scraper untuk mengakses ulasan dan informasi aplikasi dari Google Play Store. Dalam kode ini, kita menggunakan fungsi reviews_all() untuk mengambil semua ulasan dari aplikasi tersebut. Proses ini mungkin memerlukan beberapa saat untuk menyelesaikan tugasnya, terutama jika ada banyak ulasan yang perlu diambil. 
Anda juga dapat menyesuaikan bahasa ulasan (dalam contoh ini, kita menggunakan bahasa Indonesia) dan negara (dalam contoh ini, kita menggunakan Indonesia) sesuai  dengan kebutuhan. Pengguna juga dapat menentukan jumlah maksimum ulasan yang ingin diambil (dalam contoh ini, 1000 ulasan). Ulasan akan diurutkan berdasarkan relevansi, ini berarti ulasan yang dianggap paling relevan akan ditampilkan terlebih dahulu.

In [3]:
# Menyimpan ulasan dalam file CSV
import csv
 
with open('ulasan_aplikasi90.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Review'])  # Menulis header kolom
    for review in scrapreview:
        writer.writerow([review['content']])  # Menulis konten ulasan ke dalam file CSV

#### Loading Dataset
Selanjutnya, jika sudah berhasil, kita dapat melihat dataset tersebut dengan kode seperti berikut.

In [4]:
app_reviews_df = pd.DataFrame(scrapreview)
app_reviews_df.shape
app_reviews_df.head()
app_reviews_df.to_csv('ulasan_aplikasi.csv', index=False)
 
# Membuat DataFrame dari hasil scrapreview
app_reviews_df = pd.DataFrame(scrapreview)
 
# Menghitung jumlah baris dan kolom dalam DataFrame
jumlah_ulasan, jumlah_kolom = app_reviews_df.shape

Kode ini untuk mengambil ulasan aplikasi menggunakan scrapreview dan menyimpannya dalam sebuah DataFrame. Setelah itu, kita menampilkan jumlah baris dan kolom dalam dataset serta lima baris pertama untuk tinjauan cepat. Akhirnya, dataset disimpan dengan format CSV dan nama 'ulasan_aplikasi.csv' untuk digunakan dalam analisis selanjutnya.

In [5]:
# Menampilkan lima baris pertama dari DataFrame app_reviews_df
app_reviews_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,87509e9b-0076-4eb7-9c74-83ee5b8456ef,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"sangat buruk,setiap membeli kuota dengan apk i...",1,83,1.59.0,2025-01-16 23:07:40,"Hai Kak, maaf udah bikin ga nyaman ya :( Kalau...",2025-01-17 06:11:59,1.59.0
1,ff7da062-d90e-447c-afad-97a488dcf1ea,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi berat banget, suka bengong. Kode prog...",1,41,1.60.0,2025-02-01 07:23:53,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-02 03:36:51,1.60.0
2,ef265867-a243-4a97-bc6a-4209d51b2525,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Kualitas Jaringan sangat lemot dan sering terg...,1,12,1.59.0,2025-01-26 16:37:31,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-01-27 00:22:54,1.59.0
3,02cfdf8a-a35e-449c-a92e-c5c9668dd08c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Bagi kalian yang baca ini terus jaringan byu l...,1,6,1.60.0,2025-02-03 05:34:20,"Hai Kak, maaf nih terkait kendalanya, bisa cob...",2025-02-03 08:00:18,1.60.0
4,db7af5e1-2a87-4263-b88f-e45da93c1dbe,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Mohon perbaikan nya ini kenapa nge bug gini pa...,1,11,1.60.0,2025-02-01 21:39:44,"Hi, Kak. Maaf banget jadi bikin gak nyaman. Ni...",2025-02-02 03:39:54,1.60.0


Kode ini menampilkan lima baris pertama dari DataFrame app_reviews_df. DataFrame adalah struktur data tabular yang digunakan dalam pemrograman Python dan metode .head() digunakan untuk melihat beberapa baris awal dari DataFrame tersebut. Hasilnya berikut.

In [6]:
# Menampilkan informasi tentang DataFrame app_reviews_df
app_reviews_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144000 entries, 0 to 143999
Data columns (total 11 columns):
 #   Column                Non-Null Count   Dtype         
---  ------                --------------   -----         
 0   reviewId              144000 non-null  object        
 1   userName              144000 non-null  object        
 2   userImage             144000 non-null  object        
 3   content               143999 non-null  object        
 4   score                 144000 non-null  int64         
 5   thumbsUpCount         144000 non-null  int64         
 6   reviewCreatedVersion  125804 non-null  object        
 7   at                    144000 non-null  datetime64[ns]
 8   replyContent          129523 non-null  object        
 9   repliedAt             129523 non-null  datetime64[ns]
 10  appVersion            125804 non-null  object        
dtypes: datetime64[ns](2), int64(2), object(7)
memory usage: 12.1+ MB


In [7]:
# Membuat DataFrame baru (clean_df) dengan menghapus baris yang memiliki nilai yang hilang (NaN) dari app_reviews_df
clean_df = app_reviews_df.dropna()

Kode ini membuat DataFrame baru disebut clean_df dari DataFrame app_reviews_df dengan menghapus baris dengan nilai yang hilang (NaN). Metode dropna() digunakan untuk menghapus baris dengan setidaknya satu nilai yang hilang. Dengan menggunakan langkah ini, kita dapat membersihkan dataset dari baris yang tidak lengkap sehingga memastikan keberlangsungan analisis data untuk lebih konsisten.

In [8]:
# Menghapus baris duplikat dari DataFrame clean_df
clean_df = clean_df.drop_duplicates()
 
# Menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah menghapus duplikat
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = clean_df.shape

Kode ini untuk menghapus baris duplikat dari DataFrame clean_df. Dengan menggunakan metode .drop_duplicates(), baris-baris yang identik akan dihapus, menjaga hanya satu baris dari setiap entri unik. Selanjutnya, kita menghitung jumlah baris dan kolom dalam DataFrame clean_df setelah proses penghapusan duplikat.

Dengan menghapus nilai NaN dan duplikat, dataset telah dibersihkan dan siap untuk masuk ke tahapan preprocessing teks. Dataset yang bersih akan memberikan dasar solid untuk analisis teks yang akurat dan dapat diandalkan. Sekarang, kita dapat melanjutkan pada langkah-langkah preprocessing teks sesuai dengan kebutuhan analisis kita. Dengan dataset yang sudah dipersiapkan, kita dapat memperoleh wawasan berharga dari ulasan aplikasi tersebut.

### Preprocessing Text
Dalam analisis teks, langkah penting yang harus dilakukan sebelum memulai pemodelan adalah preprocessing text. Preprocessing text adalah proses membersihkan, memproses, dan mempersiapkan teks mentah agar dapat diolah lebih lanjut dengan tepat dan akurat. Langkah-langkah preprocessing ini bertujuan menghilangkan noise, mengonversi teks ke format yang konsisten, serta mengekstraksi fitur-fitur penting untuk analisis lebih lanjut.

Fungsi-fungsi yang disediakan akan sangat berguna untuk langkah-langkah preprocessing teks. Mari kita jelaskan masing-masing fungsi dengan lebih singkat.

cleaningText(text): Membersihkan teks dengan menghapus mention, hashtag, RT (retweet), tautan (link), angka, dan tanda baca. Selain itu, karakter newline diganti dengan spasi dan spasi ekstra pada awal dan akhir teks dihapus.
casefoldingText(text): Mengonversi semua karakter dalam teks menjadi huruf kecil (lowercase) untuk membuat teks menjadi seragam.
tokenizingText(text): Memecah teks menjadi daftar kata atau token. Ini membantu dalam mengurai teks menjadi komponen-komponen dasar untuk analisis lebih lanjut.
filteringText(text): Menghapus kata-kata berhenti (stopwords) dalam teks. Daftar kata-kata berhenti telah diperbarui dengan beberapa kata tambahan.
stemmingText(text): Menerapkan stemming pada teks, yakni mengurangi kata-kata menjadi bentuk dasarnya. Anda menggunakan pustaka Sastrawi untuk melakukan stemming dalam bahasa Indonesia.
toSentence(list_words): Menggabungkan daftar kata-kata menjadi sebuah kalimat

In [9]:
import re
import string
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
 
def cleaningText(text):
    text = re.sub(r'@[A-Za-z0-9]+', '', text) # menghapus mention
    text = re.sub(r'#[A-Za-z0-9]+', '', text) # menghapus hashtag
    text = re.sub(r'RT[\s]', '', text) # menghapus RT
    text = re.sub(r"http\S+", '', text) # menghapus link
    text = re.sub(r'[0-9]+', '', text) # menghapus angka
    text = re.sub(r'[^\w\s]', '', text) # menghapus karakter selain huruf dan angka
 
    text = text.replace('\n', ' ') # mengganti baris baru dengan spasi
    text = text.translate(str.maketrans('', '', string.punctuation)) # menghapus semua tanda baca
    text = text.strip(' ') # menghapus karakter spasi dari kiri dan kanan teks
    return text
 
def casefoldingText(text): # Mengubah semua karakter dalam teks menjadi huruf kecil
    text = text.lower()
    return text
 
def tokenizingText(text): # Memecah atau membagi string, teks menjadi daftar token
    text = word_tokenize(text)
    return text
 
def filteringText(text): # Menghapus stopwords dalam teks
    listStopwords = set(stopwords.words('indonesian'))
    listStopwords1 = set(stopwords.words('english'))
    listStopwords.update(listStopwords1)
    listStopwords.update(['iya','yaa','gak','nya','na','sih','ku',"di","ga","ya","gaa","loh","kah","woi","woii","woy"])
    filtered = []
    for txt in text:
        if txt not in listStopwords:
            filtered.append(txt)
    text = filtered
    return text
 
def stemmingText(text): # Mengurangi kata ke bentuk dasarnya yang menghilangkan imbuhan awalan dan akhiran atau ke akar kata
    # Membuat objek stemmer
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
 
    # Memecah teks menjadi daftar kata
    words = text.split()
 
    # Menerapkan stemming pada setiap kata dalam daftar
    stemmed_words = [stemmer.stem(word) for word in words]
 
    # Menggabungkan kata-kata yang telah distem
    stemmed_text = ' '.join(stemmed_words)
 
    return stemmed_text
 
def toSentence(list_words): # Mengubah daftar kata menjadi kalimat
    sentence = ' '.join(word for word in list_words)
    return sentence

Dengan menggunakan fungsi-fungsi ini, Anda dapat membersihkan, memproses, dan mempersiapkan teks sebelum melakukan analisis sentimen atau tugas analisis teks lainnya. Pastikan untuk memanggil fungsi-fungsi ini dengan benar sesuai dengan langkah-langkah preprocessing teks yang Anda butuhkan dalam proyek Anda.

Selanjutnya adalah penghapus kumpulan slang words atau kata-kata informal yang sering digunakan dalam percakapan sehari-hari, terutama pada media sosial atau obrolan online. Setiap kata slang memiliki padanan atau substitusi dengan kata formal atau baku. Misalnya, "abis" merupakan singkatan dari "habis", "wtb" merupakan singkatan dari "beli", dan seterusnya. Kamus ini berguna untuk membantu pemahaman dan interpretasi teks yang menggunakan bahasa informal atau slang.

In [10]:
slangwords = {"@": "di", "abis": "habis", "wtb": "beli", "masi": "masih", "wts": "jual", "wtt": "tukar", "bgt": "banget", "maks": "maksimal"}
def fix_slangwords(text):
    words = text.split()
    fixed_words = []
 
    for word in words:
        if word.lower() in slangwords:
            fixed_words.append(slangwords[word.lower()])
        else:
            fixed_words.append(word)
 
    fixed_text = ' '.join(fixed_words)
    return fixed_text

Setelah semua langkah preprocessing telah ditetapkan, langkah berikutnya adalah menerapkannya.

In [11]:
# Membersihkan teks dan menyimpannya di kolom 'text_clean'
clean_df['text_clean'] = clean_df['content'].apply(cleaningText)
 
# Mengubah huruf dalam teks menjadi huruf kecil dan menyimpannya di 'text_casefoldingText'
clean_df['text_casefoldingText'] = clean_df['text_clean'].apply(casefoldingText)
 
# Mengganti kata-kata slang dengan kata-kata standar dan menyimpannya di 'text_slangwords'
clean_df['text_slangwords'] = clean_df['text_casefoldingText'].apply(fix_slangwords)
 
# Memecah teks menjadi token (kata-kata) dan menyimpannya di 'text_tokenizingText'
clean_df['text_tokenizingText'] = clean_df['text_slangwords'].apply(tokenizingText)
 
# Menghapus kata-kata stop (kata-kata umum) dan menyimpannya di 'text_stopword'
clean_df['text_stopword'] = clean_df['text_tokenizingText'].apply(filteringText)
 
# Menggabungkan token-token menjadi kalimat dan menyimpannya di 'text_akhir'
clean_df['text_akhir'] = clean_df['text_stopword'].apply(toSentence)

#### Pelabelan
Sebelum masuk ke tahap pemodelan, langkah yang dilakukan adalah pelabelan. Pelabelan adalah proses pemberian kategori atau label pada setiap entri data berdasarkan informasi yang tersedia. Dalam konteks ini, setiap entri dataset diberikan label sentimen berdasarkan analisis teksnya. Dengan demikian, tahapan pelabelan menjadi dasar untuk proses selanjutnya dalam membangun model klasifikasi sentimen.

In [12]:
import csv
import requests
from io import StringIO
 
# Membaca data kamus kata-kata positif dari GitHub
lexicon_positive = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_positive[row[0]] = int(row[1])
        # Menambahkan kata-kata positif dan skornya ke dalam kamus lexicon_positive
else:
    print("Failed to fetch positive lexicon data")
 
# Membaca data kamus kata-kata negatif dari GitHub
lexicon_negative = dict()
 
response = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')
# Mengirim permintaan HTTP untuk mendapatkan file CSV dari GitHub
 
if response.status_code == 200:
    # Jika permintaan berhasil
    reader = csv.reader(StringIO(response.text), delimiter=',')
    # Membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma
 
    for row in reader:
        # Mengulangi setiap baris dalam file CSV
        lexicon_negative[row[0]] = int(row[1])
        # Menambahkan kata-kata negatif dan skornya dalam kamus lexicon_negative
else:
    print("Failed to fetch negative lexicon data")

Kode di atas untuk melakukan analisis sentimen pada teks berbahasa Indonesia menggunakan kamus kata-kata positif dan negatif. Pertama, kita mengirim permintaan HTTP untuk mengambil data kamus kata-kata positif dan negatif dari GitHub. 
Jika permintaan berhasil, kita membaca teks respons sebagai file CSV menggunakan pembaca CSV dengan pemisah koma. Kemudian, setiap baris dalam file CSV dibaca dan kata-kata beserta skornya dimasukkan ke kamus yang sesuai (lexicon_positive untuk kata-kata positif dan lexicon_negative untuk kata-kata negatif).

In [14]:
# Fungsi untuk menentukan polaritas sentimen dari tweet
 
def sentiment_analysis_lexicon_indonesia(text):
    #for word in text:
 
    score = 0
    # Inisialisasi skor sentimen ke 0
 
    for word in text:
        # Mengulangi setiap kata dalam teks
 
        if (word in lexicon_positive):
            score = score + lexicon_positive[word]
            # Jika kata ada dalam kamus positif, tambahkan skornya ke skor sentimen
 
    for word in text:
        # Mengulangi setiap kata dalam teks (sekali lagi)
 
        if (word in lexicon_negative):
            score = score + lexicon_negative[word]
            # Jika kata ada dalam kamus negatif, kurangkan skornya dari skor sentimen
 
    polarity=''
    # Inisialisasi variabel polaritas
 
    if (score >= 0):
        polarity = 'positive'
        # Jika skor sentimen lebih besar atau sama dengan 0, maka polaritas adalah positif
    elif (score < 0):
        polarity = 'negative'
        # Jika skor sentimen kurang dari 0, maka polaritas adalah negatif
 
    # else:
    #     polarity = 'neutral'
    # Ini adalah bagian yang bisa digunakan untuk menentukan polaritas netral jika diperlukan
 
    return score, polarity
    # Mengembalikan skor sentimen dan polaritas teks

Setelah kamus-kamus tersebut terisi, kita mendefinisikan fungsi sentiment_analysis_lexicon_indonesia yang akan menerima teks sebagai input. Dalam fungsi ini, setiap kata pada teks akan diperiksa, ada dalam kamus positif atau negatif. Jika ada, skor sentimen akan ditambahkan atau dikurangkan sesuai dengan skor kata tersebut dalam kamus. Setelah semua kata diperiksa, skor sentimen akan digunakan untuk menentukan polaritas teks, positif atau negatif. 

In [15]:
results = clean_df['text_stopword'].apply(sentiment_analysis_lexicon_indonesia)
results = list(zip(*results))
clean_df['polarity_score'] = results[0]
clean_df['polarity'] = results[1]
print(clean_df['polarity'].value_counts())

polarity
positive    64790
negative    47523
Name: count, dtype: int64


Pada DataFrame clean_df, ada 423 teks dengan polaritas negatif dan 173 teks dengan polaritas positif. Nah, setelah mendapatkan jumlah pasti dari penyebaran label dataset, selanjutnya kita akan melakukan eksplorasi jumlahnya. Dari hasil analisis, 71% dari teks memiliki polaritas negatif, sementara 29% memiliki polaritas positif. 

#### Eksplorasi Label

Lalu, bagaimana penyebaran kata-kata dalam setiap labelnya? Mari kita cek! 
Teman-teman, bagaimana pendapat Anda terkait visualisasi berikut? Berikut adalah visualisasi dari kata-kata yang sering muncul pada dataset review aplikasi By.U. Visualisasi ini menggunakan WorldCloud. WordCloud adalah representasi visual dari kata-kata yang muncul dalam teks, ketika ukurannya menunjukkan seberapa sering kata tersebut muncul. 
Nah, bagaimana jika Anda menulis pendapat dan analisis pada Forum Diskusi? Mungkin saja kita bisa melihat persepsi teman-teman yang lainnya. Kue ape kue cucur, meluncuuurr!
Di bawah ini, ada tiga visualisasi, yaitu (1) WordCloud secara general; (2) WordCloud untuk Positive Tweets Data; dan (3) Word Cloud untuk Negative Tweets Data. Berikan analisis Anda terhadap tiga gambar tersebut secara lengkap dan komprehensif! Kami tunggu di Forum Diskusi, ya. ????



#### Data Splitting dan Ekstraksi Fitur dengan TF-IDF
Nah, setelah berhasil menetapkan label untuk setiap entri dalam dataset, sekarang kita dapat melangkah ke tahap berikutnya, yaitu ekstraksi fitur dan pemisahan data. Apakah teman-teman masih ingat tentang ekstraksi fitur yang telah kita pelajari sebelumnya? Pada contoh kali ini, kita akan menggunakan metode ekstraksi fitur yang disebut TF-IDF.

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
# Pisahkan data menjadi fitur (tweet) dan label (sentimen)
X = clean_df['text_akhir']
y = clean_df['polarity']
 
# Ekstraksi fitur dengan TF-IDF
tfidf = TfidfVectorizer(max_features=200, min_df=17, max_df=0.8 )
X_tfidf = tfidf.fit_transform(X)
 
# Konversi hasil ekstraksi fitur menjadi dataframe
features_df = pd.DataFrame(X_tfidf.toarray(), columns=tfidf.get_feature_names_out())
 
# Menampilkan hasil ekstraksi fitur
features_df
 
# Bagi data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2, random_state=42)

#### Modeling
Setelah proses ekstraksi fitur dan pemisahan data, tahapan terakhir adalah melakukan pemodelan. Pada contoh ini, kita menggunakan empat algoritma yang berbeda: Naive Bayes (NB), Random Forest (RF), Logistic Regression (LR), dan Decision Tree (DT). 

In [18]:
from sklearn.naive_bayes import BernoulliNB
 
# Membuat objek model Naive Bayes (Bernoulli Naive Bayes)
naive_bayes = BernoulliNB()
 
# Melatih model Naive Bayes pada data pelatihan
naive_bayes.fit(X_train.toarray(), y_train)
 
# Prediksi sentimen pada data pelatihan dan data uji
y_pred_train_nb = naive_bayes.predict(X_train.toarray())
y_pred_test_nb = naive_bayes.predict(X_test.toarray())
 
# Evaluasi akurasi model Naive Bayes
accuracy_train_nb = accuracy_score(y_pred_train_nb, y_train)
accuracy_test_nb = accuracy_score(y_pred_test_nb, y_test)
 
# Menampilkan akurasi
print('Naive Bayes - accuracy_train:', accuracy_train_nb*100, "%")
print('Naive Bayes - accuracy_test:', accuracy_test_nb*100, "%")

Naive Bayes - accuracy_train: 78.29159710628826 %
Naive Bayes - accuracy_test: 77.92814851088457 %
